In [2]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [37]:
function load_jld2_and_translate(filepath::String)
    if split(filepath, ".")[end] != "jld2"
        println("No jld2 file: ", filepath)
        return
    end
    root = join(split(filepath, "/")[1:end-1], "/") * "/"
    file = split(filepath, "/")[end]
    data = load(filepath)
    if issetequal(Set(keys(data)),  Set(["template_mol", "template_radii", "x_init"]))
        mkpath(root * "old_format_backup/")
        template_centers = data["template_mol"]
        mv(filepath, "$(root)old_format_backup/$(file)")
        @save filepath template_centers template_radii x_init
    else
        println("The following format has not been parsed yet: ")
        println(keys(data))
    end
end


load_jld2_and_translate (generic function with 1 method)

In [38]:
folder = "../../Data/jld2/2_experimental_test/"
for file in readdir(folder)
    load_jld2_and_translate("$(folder)$(file)")
end

No jld2 file: ../../Data/jld2/2_experimental_test/.DS_Store


In [41]:
folder = "../../Data/simulations_sol_sim_format/6r7m/"
out_folder = "../../Data/collected_simulation_results/2su/6r7m/"
mkpath(out_folder)
for file in readdir(folder)
    @load "$(folder)$(file)" output_container
    id = split(file, "_")[2][1:end-5]
    template_centers = reduce(hcat,output_container.initial_state.template_centers)
    template_radii = Vector{Float64}(output_container.initial_state.radii)
    states = Vector{Vector{Float64}}([])
    x_init = Vector{Float64}([])
    for i in 1:length(output_container.initial_state.rotations)
        R = log(output_container.initial_state.rotations[i])
        T = output_container.initial_state.translations[i]
        x_init = [x_init; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
    end
    push!(states, x_init)
    n = length(output_container.initial_state.rotations)
    for step in output_container.steps
        T = step.translation
        R = log(step.rotation)
        i = step.index
        x = deepcopy(states[end])
        x[(i-1)*6 + 1: i*6] += [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]
        push!(states, x)
    end

    output = Dict{String, Any}(
        "states" => states,
        "Es" => output_container.Es,
        "αs" => output_container.αs,
        )

    input = Dict{String, Any}(
        "template_centers" => template_centers,
        "template_radii" => template_radii,
        "n_mol" => 2,
        "T" => output_container.T,
        "σ_t" => output_container.σ,
        "σ_r" => output_container.max_rotation,
        "probe_radius" => output_container.rs,
        "packing_fraction" => output_container.η,
        "overlap_jump" => output_container.overlap_jump,
        "overlap_slope" => output_container.overlap_slope,
        "delaunay_eps" => output_container.delaunay_eps
    )
    @save "$(out_folder)$(id)_6r7m.jld2" input output
end

1
2
2
2
2
1
1
2
1
2
2
1
1
1
1
1
2
2
1
2
2
1
2
2
1
2
1
1
2
2
1
1
1
1
2
1
1
2
2
1
1
1
2
2
1
1
1
1
1
2
1
2
1
2
2
1
1
2
1
2
2
1
2
1
2
2
2
1
1
2
2
1
1
1
2
2
1
1
1
2
1
1
2
2
2
1
1
2
2
1
2
1
2
2
1
2
1
2
2
1
2
2
1
2
1
2
1
1
2
2
2
1
1
2
1
1
2
1
2
2
1
1
2
1
2
1
2
2
2
2
1
1
1
2
1
2
2
1
1
1
2
1
2
1
2
1
1
2
2
1
2
1
2
2
1
1
1
1
1
2
2
2
2
2
1
2
1
2
2
1
1
2
1
2
2
1
1
2
2
2
2
2
2
2
1
1
2
1
1
2
1
2
2
2
1
2
1
2
1
1
1
1
1
2
1
1
2
1
2
2
2
1
2
2
2
2
2
2
2
2
2
2
1
1
1
1
2
2
1
1
2
1
1
2
2
2
1
2
2
2
1
2
2
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2
2
2
1
2
2
1
2
1
1
2
2
1
2
1
2
1
2
1
2
1
1
2
2
2
1
2
1
2
1
1
1
2
2
1
1
2
2
2
1
1
1
2
2
1
1
2
1
2
2
2
1
1
2
2
1
1
2
2
1
2
1
2
1
2
2
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
2
1
2
2
2
1
2
2
2
2
1
1
2
2
2
1
1
2
2
2
2
1
2
2
2
1
1
1
2
2
2
1
2
1
1
1
1
1
2
1
2
2
2
1
1
2
2
2
2
2
2
1
1
1
1
1
1
2
2
1
2
1
1
2
1
1
2
2
2
1
2
2
2
2
2
2
1
2
2
1
1
1
1
2
2
1
2
2
1
1
2
2
2
2
2
2
2
2
2
2
1
2
2
1
2
1
1
1
1
2
1
2
2
2
1
1
2
2
2
1
2
2
1
1
2
2
1
2
1
2
1
2
1
2
1
1
2
1
2
2
1
2
1
2
2
1
1
2
2
2
2
1
1
2
1
2
1
1
2
2
2
2


┌ Warning: type SolSim.DataStructures.VerboseMoleculesSimulationInfo does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type SolSim.DataStructures.Molecules does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.Point{3,Float64} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.Vec{3,Float64} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.Connectivity{Meshes.Ngon,3} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.SimpleTopology{JLD2.ReconstructedStatic{Symbol("Connectivity{Ngon,3}"), (:indices,), Tuple{Tuple{Int64, Int64, Int64

2
2
1
1
1
2
2
1
1
2
2
2
2
2
1
1
2
1
2
2
1
2
1
1
1
2
2
1
1
2
1
2
1
2
2
1
1
2
2
2
1
2
2
2
1
1
2
1
1
2
2
2
1
1
2
1
2
1
2
1
1
2
1
2
1
2
2
1
1
1
2
1
1
2
2
2
1
1
1
2
1
1
2
2
2
2
2
2
1
2
1
2
2
1
2
1
1
1
2
2
1
2
2
2
2
1
1
1
2
1
1
1
2
1
2
2
2
1
2
1
1
2
1
1
2
2
2
2
2
1
2
1
2
1
2
2
2
2
2
1
1
2
1
2
1
2
1
1
1
2
2
2
2
2
2
1
1
2
2
1
2
2
1
2
2
1
2
1
2
1
1
2
1
2
2
2
1
2
2
2
2
1
1
2
2
2
1
1
2
1
1
2
2
2
1
1
1
1
1
1
1
2
1
1
2
1
1
1
2
2
2
1
1
2
2
2
1
1
1
1
1
1
2
2
1
2
2
2
1
2
1
1
1
1
1
1
2
2
2
1
2
2
1
2
1
1
2
1
2
2
1
2
2
2
2
2
2
2
2
1
1
1
2
2
2
2
2
1
1
2
2
1
1
1
2
1
2
2
1
2
2
2
1
2
1
2
2
1
2
2
2
1
1
1
1
2
1
1
2
2
2
1
1
2
1
2
1
2
2
1
1
1
2
2
2
2
2
1
2
1
2
2
2
2
2
1
1
2
1
2
1
2
1
2
2
1
1
2
1
2
1
2
2
2
2
2
2
2
1
2
1
2
1
2
2
2
1
1
2
2
2
2
2
2
2
2
1
1
1
2
1
2
2
2
1
1
2
1
2
2
2
2
2
1
1
2
1
1
2
2
1
2
2
2
2
1
1
2
2
1
2
2
1
1
1
1
1
1
2
1
2
1
1
2
2
1
2
1
1
1
1
2
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
2
2
2
2
2
1
1
2
1
2
1
2
1
1
2
2
2
1
2
1
2
2
1
2
1
2
1
2
1
1
1
1
1
2
2
2
1
2
2
2
1
2
2
2
2
1
2
1
1
1
1
2
2
2
2
1
1
2
2
1
2
2
2


┌ Warning: type SolSim.DataStructures.VerboseMoleculesSimulationInfo does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type SolSim.DataStructures.Molecules does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.Point{3,Float64} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.Vec{3,Float64} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.Connectivity{Meshes.Ngon,3} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.SimpleTopology{JLD2.ReconstructedStatic{Symbol("Connectivity{Ngon,3}"), (:indices,), Tuple{Tuple{Int64, Int64, Int64

1
1
1
1
1
2
2
2
2
1
2
1
2
1
1
2
2
2
1
2
2
2
2
2
2
1
1
2
1
1
2
1
2
2
2
2
1
1
2
2
1
2
2
1
2
2
1
1
1
1
2
2
2
1
1
2
1
2
1
1
2
2
2
1
2
1
2
2
2
2
1
1
2
2
2
1
1
1
2
1
1
1
2
2
1
1
2
2
2
1
2
1
2
2
1
2
2
2
1
1
2
2
2
2
2
2
1
1
2
2
1
2
1
1
2
1
1
2
1
2
2
1
1
1
2
2
1
2
2
1
1
2
1
1
2
1
1
2
1
2
1
2
2
2
1
2
2
1
2
2
1
1
2
1
2
2
2
2
2
1
2
2
1
1
1
2
1
2
2
2
2
2
2
1
2
1
1
1
1
1
1
2
1
2
2
2
1
2
2
2
2
2
1
2
2
1
1
1
1
1
2
2
1
1
1
2
2
1
1
1
2
2
1
2
1
2
2
1
2
1
2
2
2
1
2
2
2
1
1
1
2
2
2
2
2
2
1
1
1
1
1
1
2
2
2
2
1
1
2
1
2
1
2
2
2
2
1
2
2
2
1
1
2
2
1
2
2
1
2
2
2
1
2
2
1
2
1
1
1
2
2
2
1
1
1
2
2
1
2
2
2
1
2
1
1
1
2
2
2
1
2
2
2
2
1
1
2
1
1
2
1
2
2
1
2
1
2
2
1
2
1
1
2
1
2
1
1
2
2
2
2
1
2
2
1
2
1
1
1
1
1
2
1
2
2
2
1
2
2
1
2
1
2
1
2
2
1
2
1
2
1
2
1
1
2
2
2
2
1
1
1
1
2
1
1
2
2
2
1
2
1
2
2
1
2
1
2
1
1
1
1
1
1
1
2
2
1
2
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
2
2
2
1
1
2
1
1
1
1
1
2
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
2
2
2
1
2
2
1
1
1
2
1
2
1
1
1
1
1
1
2
2
2
1
2
1
1
1
1
1
2
2
2
1
1
1
2
2
2
2
1
2
2
1
1
1
1
1
1
2
1
1
2
1
1
2


┌ Warning: type SolSim.DataStructures.VerboseMoleculesSimulationInfo does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type SolSim.DataStructures.Molecules does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.Point{3,Float64} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.Vec{3,Float64} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.Connectivity{Meshes.Ngon,3} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type Meshes.SimpleTopology{JLD2.ReconstructedStatic{Symbol("Connectivity{Ngon,3}"), (:indices,), Tuple{Tuple{Int64, Int64, Int64

2
2
2
2
2
2
1
1
2
2
2
2
2
1
1
1
2
2
2
2
2
2
1
1
2
1
1
1
1
1
1
1
2
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
2
1
1
2
1
1
2
2
1
2
1
1
2
2
1
2
1
1
1
1
2
1
1
2
1
1
1
2
1
1
2
1
2
1
1
2
1
1
2
2
1
1
2
2
2
1
1
2
2
2
2
1
2
2
2
1
2
1
1
1
2
2
1
2
1
1
1
1
2
1
1
1
1
2
1
2
1
2
2
1
2
1
2
1
1
2
2
1
2
1
2
2
1
2
1
1
1
2
1
1
2
2
1
2
2
1
1
2
1
2
2
1
1
1
1
2
1
2
2
2
2
2
1
1
1
1
1
1
2
1
2
2
1
2
1
1
1
1
2
2
1
2
2
1
1
2
2
1
1
2
2
1
2
2
2
1
1
2
1
2
1
1
1
2
1
2
1
2
1
1
1
2
1
1
2
2
2
1
1
2
1
2
2
2
2
2
2
1
1
1
1
1
2
1
2
2
1
1
2
1
2
2
2
2
1
1
1
2
2
2
1
2
2
2
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
1
2
2
1
1
1
2
2
2
2
2
1
2
1
2
2
2
1
1
1
2
2
2
2
2
2
1
2
2
2
2
2
2
1
1
2
2
1
2
2
1
1
2
2
2
2
2
2
2
2
2
2
1
1
2
2
1
1
1
1
2
2
2
1
2
2
1
1
1
1
1
2
2
1
2
2
1
1
1
1
2
2
1
2
1
1
1
2
1
2
2
1
1
2
1
1
1
1
1
2
1
1
1
2
2
1
2
1
1
2
2
2
2
2
1
2
1
2
1
1
1
2
2
2
1
2
2
1
2
2
1
1
1
2
2
1
1
2
2
1
2
1
2
2
2
1
2
1
2
1
1
2
2
2
2
1
2
2
2
1
2
2
1
2
1
2
1
2
1
1
2
1
1
1
2
2
2
2
1
2
1
2
2
2
2
2
1
1
1
1
2
1
1
1
2
1
2
2
2
2
1
1
1
2
1
2
1
2
1
1
1
2
2
2
2
1
2


LoadError: InterruptException: